In [39]:
from gensim import corpora, models
from gensim.models import Phrases
from gensim.models.ldamulticore import LdaMulticore
from operator import itemgetter
import re
from pymorphy2 import MorphAnalyzer
from pandas import Series

In [40]:
from nltk.corpus import names
import nltk; nltk.download('stopwords')
# NLTK Stop words
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
russian_stopwords.append('привет')
russian_stopwords.append('спасибо')
russian_stopwords.append('пожалуйста')
russian_stopwords.append('пока')
russian_stopwords.append('добрый')
russian_stopwords.append('день')
russian_stopwords.append('nan')
russian_stopwords.append('end')
russian_stopwords.append('утро')
russian_stopwords.append('ок')
russian_stopwords.append('здравствуйте')
russian_stopwords.append('мочь')
russian_stopwords.append('не')
russian_stopwords.append('работать')
russian_stopwords.append('сей')
russian_stopwords.append('пора')
russian_stopwords.append('очень')
russian_stopwords.append('проблема')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sych_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [73]:
class LdaPredictor():
    
    def __init__(self, lda_path, dict_path, bigram_path, trigram_path):
        """
        lda_path - путь к lda модели
        dict_path - путь к словарю 
        bigram_path - путь к биграммам
        trigram_path - путь к триграммам
        
        param: lda_path str
        param: dict_path str
        param: bigram_path str
        param: trigram_path str
        """
        self.dictionary = corpora.Dictionary.load(dict_path)
        self.lda = LdaMulticore.load(lda_path)
        self.bigram_path = bigram_path
        self.trigram_path = trigram_path
        
    def to_lemmatize2(self, text):
        all_word_str = " ".join(text)
        all_word_list = all_word_str.split()
        all_unique_word = Series(all_word_list).unique()
        lemmatized_word_dict = {}
        lemmatizer = MorphAnalyzer()
        for word in all_unique_word:
            lemmatized_word_dict[word] = lemmatizer.normal_forms(word)[0]
        text = ' '.join([lemmatized_word_dict[word] for word in text])
        return text, all_unique_word
        
    def clean(self, text):
        deleted_symols = '[\\\\\'[\]!"$%&()*+,-./:;<=>?@^_`{|}~«»\n]'
        text = re.sub(deleted_symols, ' ', text)
        
        text = ' '.join([elem for elem in str(text).split(' ') if elem.isdigit() == False])
        
        text = text.lower()
        text = [token for token in text.split() if token not in russian_stopwords]
        #text = [token for token in text if token]
        text, _ = self.to_lemmatize2(text)
        
        return ' '.join(text)
    
    def bigram(self, text):
        bigram = Phrases.load(self.bigram_path)
        trigram = Phrases.load(self.trigram_path)
        text_clean = text
        for idx in range(len(text_clean)):
            for token in bigram[text_clean[idx]]:
                if '_' in token:
                    text_clean[idx].append(token)
            for token in trigram[text_clean[idx]]:
                if '_' in token:
                    text_clean[idx].append(token)
        return text_clean
    
    def predict(self, text):
        clean_text = self.clean(text).split()
        print(clean_text)
        bigram = self.bigram([clean_text])
        new_review_bow = self.dictionary.doc2bow(bigram[0])
        new_review_lda = self.lda[new_review_bow]
        return sorted(new_review_lda, reverse=True, key=itemgetter(1))

In [74]:
lda_path = "./model/best_model.lda"
dict_path = "./model/dictionary.dict"
bigram_path = "./model/bigram.phs"
trigram_path = "./model/trigram.phs"
lda = LdaPredictor(lda_path, dict_path,  bigram_path, trigram_path)

In [75]:
lda.lda.show_topics()

[(4,
  '0.032*"поставщик" + 0.016*"просить_помочь" + 0.015*"подтверждение" + 0.012*"ооо" + 0.012*"документ" + 0.011*"сделать" + 0.010*"видеть" + 0.009*"нужно" + 0.008*"перемещение" + 0.008*"1с"'),
 (2,
  '0.019*"касса" + 0.010*"эник" + 0.009*"пз" + 0.009*"год" + 0.008*"договор" + 0.008*"аналитик" + 0.008*"магазин" + 0.007*"ок" + 0.007*"заполнить" + 0.007*"инна"'),
 (6,
  '0.036*"удалённый" + 0.036*"рабочий_стол" + 0.033*"удалённый_доступ" + 0.031*"рабочий" + 0.027*"доступ" + 0.027*"подключиться" + 0.023*"компьютер" + 0.023*"стол" + 0.020*"зайти" + 0.017*"удалённый_рабочий"'),
 (0,
  '0.020*"база" + 0.015*"висеть" + 0.013*"очень" + 0.012*"программа" + 0.011*"сервер" + 0.010*"перезагрузить" + 0.010*"зайти" + 0.009*"заработать" + 0.009*"долго" + 0.008*"минута"'),
 (5,
  '0.051*"выдавать_ошибка" + 0.030*"выдавать" + 0.020*"фото_—" + 0.015*"ерп" + 0.015*"сервис_деск" + 0.014*"личный_кабинет" + 0.011*"фото" + 0.010*"ар00" + 0.010*"помочь" + 0.010*"вечер"'),
 (1,
  '0.023*"принять" + 0.015*"в

In [76]:
text = "При открытии формирования приходной накладной  всплывает данная ошибка. На всех компах С9  не можем поставить палеты на приход"
predict = lda.predict(text)
print(predict)

['о', 'т', 'к', 'р', 'ы', 'т', 'и', 'е', 'ф', 'о', 'р', 'м', 'и', 'р', 'о', 'в', 'а', 'н', 'и', 'е', 'п', 'р', 'и', 'х', 'о', 'д', 'н', 'ы', 'й', 'н', 'а', 'к', 'л', 'а', 'д', 'н', 'ы', 'й', 'в', 'с', 'п', 'л', 'ы', 'в', 'а', 'т', 'ь', 'д', 'а', 'т', 'ь', 'о', 'ш', 'и', 'б', 'к', 'а', 'к', 'о', 'м', 'п', 'с', '9', 'м', 'о', 'ч', 'ь', 'п', 'о', 'с', 'т', 'а', 'в', 'и', 'т', 'ь', 'п', 'а', 'л', 'е', 'т', 'а', 'п', 'р', 'и', 'х', 'о', 'д']
[(2, 0.34329468), (7, 0.123350665), (1, 0.104602076), (4, 0.05018462), (10, 0.049427148), (11, 0.049121812), (9, 0.048453752), (5, 0.047204766), (0, 0.046743352), (6, 0.046015613), (3, 0.045885153), (8, 0.045716353)]
